In [1]:
!pip install konlpy #형태로 분리 위한 라이브러리
!pip install jamo #자모 분리 위한 라이브러리

     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 448 kB 42.7 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#엑셀파일로부터 데이터프레임 생성, 불필요한 요소 제거

In [3]:
df = pd.read_excel('dataset.xlsx')
df = df.iloc[:,0:2]

#문장과 감정 (X, Y) 데이터를 분리

In [4]:
df_sentence =df['Sentence']
df_emotion = df['Emotion']

In [5]:
df_sentence.head(),df_emotion.head()

(0    언니 동생으로 부르는게 맞는 일인가요..??
 1                그냥 내 느낌일뿐겠지?
 2              아직너무초기라서 그런거죠?
 3               유치원버스 사고 낫다던데
 4                 근데 원래이런거맞나요
 Name: Sentence, dtype: object, 0    공포
 1    공포
 2    공포
 3    공포
 4    공포
 Name: Emotion, dtype: object)

#df_emotion을 학습을 위해 원-핫 인코딩

In [6]:
df_emotion = pd.get_dummies(df_emotion)

In [7]:
df_emotion

,공포,놀람,분노,슬픔,중립,행복,혐오
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...
38589,0,0,0,0,0,0,1
38590,0,0,0,0,0,0,1
38591,0,0,0,0,0,0,1
38592,0,0,0,0,0,0,1


#df_sentence를 학습을 위해 토큰화 이전에 자모, 형태소로 각각 분리


In [8]:
#분리 전 df-->list형으로 포맷 변경
array_sentence = np.array(df_sentence.to_list())

In [9]:
array_sentence[0]

'언니 동생으로 부르는게 맞는 일인가요..??'

In [10]:
array_sentence_syllable = array_sentence

In [11]:
array_sentence_syllable[0],array_sentence[0]

('언니 동생으로 부르는게 맞는 일인가요..??', '언니 동생으로 부르는게 맞는 일인가요..??')

In [12]:
#음절(자모)분리
from jamo import h2j, j2hcj

for i, document in enumerate(array_sentence_syllable):
    clean_words = []
    for word in j2hcj(h2j(document)): 
        clean_words.append(word)
    print(clean_words)
    document = ' '.join(clean_words)
    print(document)
    array_sentence_syllable[i] = document

Streaming output truncated to the last 5000 lines.
['ㄲ', 'ㅏ', 'ㅁ', 'ㅃ', 'ㅏ', 'ㄱ', 'ㅇ', 'ㅣ', ' ', 'ㅇ', 'ㅏ', 'ㄴ', ' ', 'ㅋ', 'ㅕ', 'ㅁ', 'ㅕ', 'ㄴ', ' ', 'ㅂ', 'ㅓ', 'ㅁ', 'ㅊ', 'ㅣ', 'ㄱ', 'ㄱ', 'ㅡ', 'ㅁ', ' ', 'ㅈ', 'ㅗ', 'ㅁ', ' ', 'ㅅ', 'ㅔ', 'ㄱ', 'ㅔ', ' ', 'ㅁ', 'ㅜ', 'ㄹ', 'ㄹ', 'ㅕ', 'ㄹ', 'ㅏ']
ㄲ ㅏ ㅁ ㅃ ㅏ ㄱ ㅇ ㅣ   ㅇ ㅏ ㄴ   ㅋ ㅕ ㅁ ㅕ ㄴ   ㅂ ㅓ ㅁ ㅊ ㅣ ㄱ ㄱ ㅡ ㅁ   ㅈ ㅗ ㅁ   ㅅ ㅔ ㄱ ㅔ   ㅁ ㅜ ㄹ ㄹ ㅕ ㄹ ㅏ
['ㅇ', 'ㅣ', 'ㄹ', 'ㅐ', 'ㄴ', 'ㄷ', 'ㅡ', 'ㅇ', 'ㅗ', 'ㅅ', ' ', 'ㅂ', 'ㅜ', 'ㄹ', 'ㅁ', 'ㅐ', 'ㅎ', 'ㅐ', 'ㅇ', 'ㅑ', 'ㄱ', 'ㅔ', 'ㅆ', 'ㄴ', 'ㅔ', 'ㅇ', 'ㅛ', '.']
ㅇ ㅣ ㄹ ㅐ ㄴ ㄷ ㅡ ㅇ ㅗ ㅅ   ㅂ ㅜ ㄹ ㅁ ㅐ ㅎ ㅐ ㅇ ㅑ ㄱ ㅔ ㅆ ㄴ ㅔ ㅇ ㅛ .
['ㅁ', 'ㅝ', ' ', 'ㅋ', 'ㅡ', 'ㄴ', 'ㅇ', 'ㅑ', 'ㄱ', 'ㅈ', 'ㅓ', 'ㅁ', ' ', 'ㅈ', 'ㅏ', 'ㅂ', 'ㅎ', 'ㅣ', 'ㄴ', 'ㄱ', 'ㅓ', ' ', 'ㅇ', 'ㅣ', 'ㅆ', 'ㅇ', 'ㅡ', 'ㅅ', 'ㅣ', 'ㄴ', 'ㄱ', 'ㅏ', ' ', 'ㅇ', 'ㅓ', 'ㅇ', 'ㅣ', 'ㄱ', 'ㅏ', ' ', 'ㅇ', 'ㅓ', 'ㅄ', 'ㄴ', 'ㅔ']
ㅁ ㅝ   ㅋ ㅡ ㄴ ㅇ ㅑ ㄱ ㅈ ㅓ ㅁ   ㅈ ㅏ ㅂ ㅎ ㅣ ㄴ ㄱ ㅓ   ㅇ ㅣ ㅆ ㅇ ㅡ ㅅ ㅣ ㄴ ㄱ ㅏ   ㅇ ㅓ ㅇ ㅣ ㄱ ㅏ   ㅇ ㅓ ㅄ ㄴ ㅔ
['ㄱ', 'ㅡ', 'ㄴ', 'ㅔ', 'ㅇ', 'ㅑ', ' ', 'ㄴ', 'ㅏ', 'ㄷ', 'ㅐ', 'ㅈ', 'ㅣ', 'ㅁ', 'ㅏ', 'ㄹ', 'ㄱ', 'ㅗ', ' ', 'ㅊ', 'ㅕ', ' ', 'ㄴ', 'ㅐ',

In [13]:
array_sentence_syllable[0]

'ㅇ ㅓ ㄴ ㄴ ㅣ   ㄷ ㅗ ㅇ ㅅ ㅐ ㅇ ㅇ ㅡ ㄹ ㅗ   ㅂ ㅜ ㄹ ㅡ ㄴ ㅡ ㄴ ㄱ ㅔ   ㅁ ㅏ ㅈ ㄴ ㅡ ㄴ   ㅇ ㅣ ㄹ ㅇ ㅣ ㄴ ㄱ ㅏ ㅇ ㅛ . . ? ?'

#토큰화 및 정수화

In [14]:
array_sentence[0]

'ㅇ ㅓ ㄴ ㄴ ㅣ   ㄷ ㅗ ㅇ ㅅ ㅐ ㅇ ㅇ ㅡ ㄹ ㅗ   ㅂ ㅜ ㄹ ㅡ ㄴ ㅡ ㄴ ㄱ ㅔ   ㅁ ㅏ ㅈ ㄴ ㅡ ㄴ   ㅇ ㅣ ㄹ ㅇ ㅣ ㄴ ㄱ ㅏ ㅇ ㅛ . . ? ?'

In [15]:
array_sentence_syllable[0]

'ㅇ ㅓ ㄴ ㄴ ㅣ   ㄷ ㅗ ㅇ ㅅ ㅐ ㅇ ㅇ ㅡ ㄹ ㅗ   ㅂ ㅜ ㄹ ㅡ ㄴ ㅡ ㄴ ㄱ ㅔ   ㅁ ㅏ ㅈ ㄴ ㅡ ㄴ   ㅇ ㅣ ㄹ ㅇ ㅣ ㄴ ㄱ ㅏ ㅇ ㅛ . . ? ?'

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer() #토큰 객체 생성, num_word = 빈도수 상위 100개의 단어를 토큰화 한다는 뜻, default=-1(전체)

In [17]:
'''
#형태소+자모 따로였던 리스트를 하나로 병합시키는 함수
def merge_list(*args, fill_value = None):
  max_length = max([len(lst) for lst in args])
  merged = []
  for i in range(max_length):
    merged.append([
      args[k][i] if i < len(args[k]) else fill_value for k in range(len(args))
    ])
  return merged
  '''

'\n#형태소+자모 따로였던 리스트를 하나로 병합시키는 함수\ndef merge_list(*args, fill_value = None):\n  max_length = max([len(lst) for lst in args])\n  merged = []\n  for i in range(max_length):\n    merged.append([\n      args[k][i] if i < len(args[k]) else fill_value for k in range(len(args))\n    ])\n  return merged\n  '

In [18]:
'''
merged_array = merge_list(array_sentence_morphs,array_sentence_syllable)
#list(map(list.__add__, array_sentence_morphs, array_sentence_syllable))
'''

'\nmerged_array = merge_list(array_sentence_morphs,array_sentence_syllable)\n#list(map(list.__add__, array_sentence_morphs, array_sentence_syllable))\n'

In [19]:
#merged_array[0]

In [20]:
tokenizer.fit_on_texts(array_sentence_syllable) #-->morphs 와 syllable을 정수화해야하는데, 하나의 리스트밖에 적용되지 않음, 위의 리스트는 오류발생. 해결 필요

In [21]:
tokenizer.word_index

{"'": 70,
 '0': 40,
 '1': 42,
 '2': 43,
 '3': 44,
 '4': 47,
 '5': 45,
 '6': 51,
 '7': 48,
 '8': 52,
 '9': 53,
 'a': 63,
 'b': 61,
 'c': 59,
 'd': 75,
 'e': 77,
 'f': 81,
 'g': 62,
 'h': 84,
 'i': 65,
 'j': 76,
 'k': 64,
 'l': 66,
 'm': 69,
 'n': 73,
 'o': 56,
 'p': 71,
 'r': 82,
 's': 54,
 't': 57,
 'u': 85,
 'v': 72,
 'w': 86,
 'x': 79,
 'y': 83,
 'z': 95,
 '·': 90,
 '×': 91,
 '‘': 96,
 '’': 97,
 '‥': 87,
 '…': 88,
 '→': 98,
 '≪': 104,
 '≫': 103,
 '⊙': 107,
 '△': 106,
 '▽': 113,
 '★': 92,
 '☆': 93,
 '♡': 55,
 '♥': 67,
 '♪': 112,
 '♬': 94,
 '《': 114,
 '》': 115,
 'ㄱ': 4,
 'ㄲ': 27,
 'ㄳ': 89,
 'ㄴ': 3,
 'ㄵ': 80,
 'ㄶ': 38,
 'ㄷ': 9,
 'ㄸ': 32,
 'ㄹ': 6,
 'ㄺ': 46,
 'ㄻ': 58,
 'ㄼ': 74,
 'ㄾ': 116,
 'ㅀ': 49,
 'ㅁ': 11,
 'ㅂ': 16,
 'ㅃ': 36,
 'ㅄ': 37,
 'ㅅ': 12,
 'ㅆ': 20,
 'ㅇ': 1,
 'ㅈ': 13,
 'ㅉ': 33,
 'ㅊ': 22,
 'ㅋ': 21,
 'ㅌ': 24,
 'ㅍ': 25,
 'ㅎ': 14,
 'ㅏ': 2,
 'ㅐ': 18,
 'ㅑ': 28,
 'ㅒ': 50,
 'ㅓ': 7,
 'ㅔ': 17,
 'ㅕ': 19,
 'ㅖ': 39,
 'ㅗ': 10,
 'ㅘ': 26,
 'ㅙ': 41,
 'ㅚ': 30,
 'ㅛ': 23,
 'ㅜ': 15,
 'ㅝ': 31,
 'ㅞ': 60

In [22]:
#정수화된 토큰을 이용해 리스트를 정수화
sequenced_syllable = tokenizer.texts_to_sequences(array_sentence_syllable)

#정수화된 리스트 데이터들의 길이를 맞추기 위해 zero-padding 진행

In [23]:
padded_syllable = pad_sequences(sequenced_syllable, padding='pre', maxlen=300)
                #자모는 이론상 morphs의 3배 길이(초,중,종성) 이니 상단 maxlen*3으로 설정해보자

#전처리가 끝난 데이터를 train,test 셋으로 분리

In [24]:
train_x, test_x, train_y, test_y =train_test_split(padded_syllable,df_emotion)

In [25]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((28945, 300), (9649, 300), (28945, 7), (9649, 7))

In [26]:
#LSTM 학습을 위해 3D tensor reshape
train_x = train_x.reshape(28945,300,1)
test_x = test_x.reshape(9649,300,1)

#모델 설계 (2개의 input(morphs, syllable)을 사용하는 multi-input CNN 기반으로 제작예정)
#->토크나이즈의 한계로 형태소 RNN, 자모 RNN 2개 모델 생성 후 두개의 결과값을 비교해봄
* 1개 인풋
* 7개 아웃풋(감정의 종류:7개)

In [27]:
from tensorflow.keras import layers,models

In [28]:
model = models.Sequential()

In [29]:
model.add(layers.Embedding(input_dim = 9000000,
                           output_dim=1,
                           input_length=100))
model.add(layers.LSTM(units=32, return_sequences=True))
model.add(layers.GRU(units=16))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(units=7, activation='sigmoid'))

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 1)            9000000   
_________________________________________________________________
lstm (LSTM)                  (None, 100, 32)           4352      
_________________________________________________________________
gru (GRU)                    (None, 16)                2400      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 7)                 119       
Total params: 9,006,871
Trainable params: 9,006,871
Non-trainable params: 0
_________________________________________________________________


# 모델 학습

In [31]:
model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])

#모델 테스트

In [32]:
#결과값을 벡터로 표현할 수 있는가?
#1번째*acc 2번째*acc 3번째 
model.fit(train_x, train_y, batch_size=2000, epochs=1000, validation_split=0.2)

Epoch 1/1000
12/12 [==============================] - 12s 358ms/step - loss: 0.2443 - accuracy: 0.1400 - val_loss: 0.2343 - val_accuracy: 0.1375
Epoch 2/1000
12/12 [==============================] - 3s 252ms/step - loss: 0.2165 - accuracy: 0.1408 - val_loss: 0.1642 - val_accuracy: 0.1375
Epoch 3/1000
12/12 [==============================] - 3s 249ms/step - loss: 0.1653 - accuracy: 0.1423 - val_loss: 0.1288 - val_accuracy: 0.1375
Epoch 4/1000
12/12 [==============================] - 3s 252ms/step - loss: 0.1531 - accuracy: 0.1461 - val_loss: 0.1253 - val_accuracy: 0.1375
Epoch 5/1000
12/12 [==============================] - 3s 251ms/step - loss: 0.1468 - accuracy: 0.1431 - val_loss: 0.1238 - val_accuracy: 0.1375
Epoch 6/1000
12/12 [==============================] - 3s 254ms/step - loss: 0.1427 - accuracy: 0.1456 - val_loss: 0.1236 - val_accuracy: 0.1375
Epoch 7/1000
12/12 [==============================] - 3s 251ms/step - loss: 0.1405 - accuracy: 0.1439 - val_loss: 0.1237 - val_accuracy

#학습된 모델 평가

In [33]:
model.evaluate(test_x,test_y)

302/302 [==============================] - 12s 39ms/step - loss: 0.1118 - accuracy: 0.3238


[0.11179563403129578, 0.3237641155719757]